<a href="https://colab.research.google.com/github/ravidu-hevaganinge/AI_Scribe/blob/MedGPT/rule_based_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!python -m spacy download en_core_web_sm

2024-03-19 15:09:21.247633: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 15:09:21.247700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 15:09:21.250148: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 15:09:23.642413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.8 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.7.1
    Uninstalling en-core-web-sm-3.7.1:
      Successfully uninstall

In [ ]:
!pip install medspacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.0/580.0 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.3/408.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/wyim/aci-bench.git
import pandas as pd
!cd aci-bench/data/src_experiment_data

# training datasets
training_data_aci = pd.read_csv("/content/aci-bench/data/src_experiment_data/train_aci_asrcorr.csv")
training_data_aci_meta = pd.read_csv("/content/aci-bench/data/src_experiment_data/train_aci_asrcorr_metadata.csv")

Cloning into 'aci-bench'...
remote: Enumerating objects: 528, done.
remote: Counting objects: 100% (528/528), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 528 (delta 357), reused 488 (delta 328), pack-reused 0
Receiving objects: 100% (528/528), 8.98 MiB | 18.80 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [ ]:
transcript = training_data_aci["dialogue"][0]
# print(transcript)

In [ ]:
import medspacy
import spacy
from medspacy.sentence_splitting import PyRuSHSentencizer
from medspacy.custom_tokenizer import create_medspacy_tokenizer
from medspacy.visualization import visualize_ent, visualize_dep
from medspacy.ner import TargetRule

In [ ]:
nlp = medspacy.load(medspacy_enable=["medspacy_pyrush"])

In [ ]:
target_matcher = nlp.add_pipe("medspacy_target_matcher")

In [ ]:
target_rules = [
    TargetRule(literal="bilateral carpal tunnel syndrome", category="PROBLEM"),
    TargetRule("numbness", "PROBLEM"),
    TargetRule("emg", "TREATMENT"),
    TargetRule("surgical intervention", "TREATMENT"),
    TargetRule("colon cancer", "PROBLEM"),
    TargetRule("metastasis", "PROBLEM"),

]

In [ ]:
target_matcher.add(target_rules)

In [ ]:
medspacy_tokenizer = create_medspacy_tokenizer(nlp)
nlp.tokenizer = medspacy_tokenizer

In [ ]:
nlp.add_pipe("medspacy_pyrush")

In [ ]:
nlp.pipe_names

['medspacy_pyrush']

In [ ]:
doc = nlp(transcript)

In [ ]:
print(doc.ents)

(numbness, numbness, numbness, numbness, bilateral carpal tunnel syndrome, emg, surgical intervention)


In [ ]:
for ent in doc.ents:
  print(ent, ent.label_, ent._.target_rule, sep="  |  ")
  print()

numbness  |  PROBLEM  |  TargetRule(literal="numbness", category="PROBLEM", pattern=None, attributes=None, on_match=None)

numbness  |  PROBLEM  |  TargetRule(literal="numbness", category="PROBLEM", pattern=None, attributes=None, on_match=None)

numbness  |  PROBLEM  |  TargetRule(literal="numbness", category="PROBLEM", pattern=None, attributes=None, on_match=None)

numbness  |  PROBLEM  |  TargetRule(literal="numbness", category="PROBLEM", pattern=None, attributes=None, on_match=None)

bilateral carpal tunnel syndrome  |  PROBLEM  |  TargetRule(literal="bilateral carpal tunnel syndrome", category="PROBLEM", pattern=None, attributes=None, on_match=None)

emg  |  TREATMENT  |  TargetRule(literal="emg", category="TREATMENT", pattern=None, attributes=None, on_match=None)

surgical intervention  |  TREATMENT  |  TargetRule(literal="surgical intervention", category="TREATMENT", pattern=None, attributes=None, on_match=None)



In [ ]:
sentences = list(doc.sents)
for sentence in sentences:
  print(sentence)
  print('----'*10)

[doctor] hey george how are you today i understand you're here for some numbness and tingling in your fingers and some pain in your wrist
[patient] right my left wrist and hand has been bothering me probably for a few months now with pain and numbness
[doctor] okay and you said that's been ongoing for several months do you know what caused this type of pain or is it just something that started slowly or
[patient] it just kinda started on it's own it i notice it mostly at night
[doctor] okay
[patient] sometimes it will i'll wake up and my hands asleep and i got ta shake it out
[doctor] shake it out and okay
[patient] and then some
[doctor] what kind of work do you do
[patient] i do yard work
[doctor] yard work
[patient] landscaping landscaping
[doctor] landscaping okay so a lot of raking a lot of digging so a lot of repetitive type movements
[patient] yeah it's pretty heavy labor but it's yeah the same thing day in and day out
[doctor] okay okay just a couple questions for you you did s

In [ ]:
nlp2 = medspacy.load("en_core_web_sm")
nlp2.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'medspacy_pyrush',
 'medspacy_target_matcher',
 'medspacy_context']

In [ ]:
context = nlp.get_pipe("medspacy_context")

In [ ]:
for rule in context.rules[:10]:
  print()
  print(rule)


ConTextRule(literal='absence of', category='NEGATED_EXISTENCE', pattern=None, direction='FORWARD')

ConTextRule(literal='adequate to rule out', category='NEGATED_EXISTENCE', pattern=[{'LOWER': {'IN': ['adequate', 'sufficient']}}, {'LOWER': 'to'}, {'LOWER': 'rule'}, {'LOWER': {'IN': ['him', 'her', 'them', 'patient', 'pt']}, 'OP': '?'}, {'LOWER': 'out'}, {'LOWER': {'IN': ['against', 'for']}, 'OP': '?'}], direction='FORWARD')

ConTextRule(literal='adequate to rule the patient out', category='NEGATED_EXISTENCE', pattern=[{'LOWER': {'IN': ['adequate', 'sufficient']}}, {'LOWER': 'to'}, {'LOWER': 'rule'}, {'LOWER': 'the'}, {'LOWER': {'IN': ['patient', 'pt']}}, {'LOWER': 'out'}, {'LOWER': {'IN': ['against', 'for']}, 'OP': '?'}], direction='FORWARD')

ConTextRule(literal='any other', category='NEGATED_EXISTENCE', pattern=None, direction='FORWARD')

ConTextRule(literal='apart from', category='NEGATED_EXISTENCE', pattern=[{'LOWER': 'apart'}, {'LOWER': {'IN': ['for', 'from']}}], direction='TERMIN